In [ ]:
library(latex2exp)
library(igraph)
library(ggplot2)
library(pracma)
library(scales)

In [ ]:
library(data.table)

### Question 18

In [ ]:
# The answer is the number of .circle files in gplus.tar.gz with >=2 lines. 

### Question 19

In [ ]:
path = "C:\\Users\\andersons1\\OneDrive - Anglo American\\Desktop\\ECE 232\\ECENGR232E-80\\ECENGR232E-80\\project_2\\gplus_combined.txt.gz"
g_gpe = as.matrix(read.table(path, header = FALSE, sep = " ")) + 1
g_gp = graph_from_edgelist(g_gpe, directed = FALSE)

In [ ]:
node_ids <- c(109327480479767108490,115625564993990145546,101373961279443806744)

g1 = make_ego_graph(g_gp, 1, nodes=V(g_gp)[node_ids[1]])
g2 = make_ego_graph(g_gp, 1, nodes=V(g_gp)[node_ids[2]])
g3 = make_ego_graph(g_gp, 1, nodes=V(g_gp)[node_ids[3]])

plot(degree.distribution(g1,mode="in"),
     col='red',
     main='v1 in-degree distribution',
     xlab="Node In-Degree",
     ylab="Degree Probability")

dev.copy(png,'images\\question19_v1_in.png')
dev.off()

plot(degree.distribution(g2,mode="in"),
     col='red',
     main='v2 in-degree distribution',
     xlab="Node In-Degree",
     ylab="Degree Probability")

dev.copy(png,'images\\question19_v2_in.png')
dev.off()

plot(degree.distribution(g3,mode="in"),
     col='red',
     main='v3 in-degree distribution',
     xlab="Node In-Degree",
     ylab="Degree Probability")

dev.copy(png,'images\\question19_v3_in.png')
dev.off()

plot(degree.distribution(g1,mode="out"),
     col='red',
     main='v1 out-degree distribution',
     xlab="Node In-Degree",
     ylab="Degree Probability")

dev.copy(png,'images\\question19_v1_out.png')
dev.off()

plot(degree.distribution(g2,mode="out"),
     col='red',
     main='v2 out-degree distribution',
     xlab="Node In-Degree",
     ylab="Degree Probability")

dev.copy(png,'images\\question19_v2_out.png')
dev.off()

plot(degree.distribution(g3,mode="out"),
     col='red',
     main='v3 out-degree distribution',
     xlab="Node In-Degree",
     ylab="Degree Probability")

dev.copy(png,'images\\question19_v3_out.png')
dev.off()

### Question 20

In [ ]:
#compute communities
c1 = cluster_walktrap(g1)
c2 = cluster_walktrap(g2)
c3 = cluster_walktrap(g3)

#compute modularities
m1 = modularity(c1)
m2 = modularity(c2)
m3 = modularity(c3)

#plot community structure
plot(c1,g1,layout=layout_with_kk(g1))
plot(c2,g2,layout=layout_with_kk(g2))
plot(c3,g3,layout=layout_with_kk(g3))

### Question 22